In [200]:
import pandas as pd
import json
import math

In [201]:
def get_pythagoras(x, y):
    return math.sqrt(x ** 2 + y ** 2)
diffPath = 'accsaber-maps/1a2f6 (Digital World - SlimyBlob)/HardStandard.dat' #Replace with your difficulty file's path
infoPath = 'accsaber-maps/1a2f6 (Digital World - SlimyBlob)/Info.dat' #Replace with your info file's path
with open(diffPath) as diff_json_data:
    diffData = json.load(diff_json_data)

with open(infoPath) as info_json_data:
    infoData = json.load(info_json_data)

In [202]:
#ADD IMPLEMENTATION FOR NO BPM CHANGES LATER



#Parse BPM changes if they exist
initialBPM = infoData.get('_beatsPerMinute')
bpmChangesDict = diffData.get('_customData').get('_BPMChanges')
df_BPMChanges = pd.DataFrame(bpmChangesDict)
df_BPMChanges

""


In [203]:
df = pd.DataFrame(diffData['_notes'])
df['_yCenter'] = df.loc[:, ('_lineLayer')].apply(lambda x: 1 + x * 0.55)
df['_xCenter'] = df.loc[:, ('_lineIndex')].apply(lambda x: -0.9 + x * 0.6)

#Add bpm column
df['_bpm'] = initialBPM
for i in range(len(df)):
    currentTime = df.loc[i, '_time']
    currentRow = 0
    for j in range(len(df_BPMChanges)):
        if currentTime >= df_BPMChanges.loc[j, '_time']:
            df['_bpm'] = df_BPMChanges.loc[j, '_time']
   

        
left = (df[df['_type'] == 0]) #All left handed notes
right = (df[df['_type'] == 1]) #All right handed notes

left['_xMovement'] = left.loc[:, ['_xCenter']].diff().fillna(0)
left['_yMovement'] = left.loc[:, ['_yCenter']].diff().fillna(0)
left['_totMovement'] = left.apply(lambda x: get_pythagoras(x['_xMovement'], x['_yMovement']), axis=1).fillna(0) 
left['_angleChange'] = left.apply(lambda x: math.atan(x['_yMovement']/x['_xMovement']), axis=1)
left['_timeChange'] = left.loc[:, ['_time']].diff().fillna(0)

right['_xMovement'] = right.loc[:, ['_xCenter']].diff().fillna(0)
right['_yMovement'] = right.loc[:, ['_yCenter']].diff().fillna(0)
right['_totMovement'] = right.apply(lambda x: get_pythagoras(x['_xMovement'], x['_yMovement']), axis=1).fillna(0)
right['_angleChange'] = right.apply(lambda x: math.atan(x['_yMovement']/x['_xMovement']), axis=1)
right['_timeChange'] = right.loc[:, ['_time']].diff().fillna(0)

average_angle = (left['_angleChange'].mean() + right['_angleChange'].mean()) / 2

/var/folders/hx/8pb__z8x49q872rhs7wcd7_m0000gn/T/ipykernel_2037/1679518073.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left['_xMovement'] = left.loc[:, ['_xCenter']].diff().fillna(0)
/var/folders/hx/8pb__z8x49q872rhs7wcd7_m0000gn/T/ipykernel_2037/1679518073.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left['_yMovement'] = left.loc[:, ['_yCenter']].diff().fillna(0)
/var/folders/hx/8pb__z8x49q872rhs7wcd7_m0000gn/T/ipykernel_2037/1679518073.py:21: SettingWithCopyWarning: 
A value is trying to 

In [204]:
df

,_time,_lineIndex,_lineLayer,_type,_cutDirection,_yCenter,_xCenter,_bpm
0,21.0,1,0,0,1,1.00,-0.3,150
1,21.0,2,0,1,1,1.00,0.3,150
2,22.0,3,1,1,5,1.55,0.9,150
3,22.5,0,1,0,4,1.55,-0.9,150
4,23.0,1,0,1,6,1.00,-0.3,150
...,...,...,...,...,...,...,...,...
668,483.5,1,2,0,0,2.10,-0.3,150
669,484.0,3,0,1,1,1.00,0.9,150
670,485.0,0,0,0,1,1.00,-0.9,150
671,485.0,0,1,0,1,1.55,-0.9,150


In [205]:
left

,_time,_lineIndex,_lineLayer,_type,_cutDirection,_yCenter,_xCenter,_bpm,_xMovement,_yMovement,_totMovement,_angleChange,_timeChange
0,21.0,1,0,0,1,1.00,-0.3,150,0.0,0.00,0.000000,NaN,0.0
3,22.5,0,1,0,4,1.55,-0.9,150,-0.6,0.55,0.813941,-0.741947,1.5
5,23.5,2,0,0,7,1.00,0.3,150,1.2,-0.55,1.320038,-0.429762,1.0
7,24.5,1,2,0,0,2.10,-0.3,150,-0.6,1.10,1.252996,-1.071450,1.0
9,25.5,0,0,0,1,1.00,-0.9,150,-0.6,-1.10,1.252996,1.071450,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,481.0,0,2,0,0,2.10,-0.9,150,0.0,0.55,0.550000,1.570796,0.0
666,482.5,2,0,0,7,1.00,0.3,150,1.2,-1.10,1.627882,-0.741947,1.5
668,483.5,1,2,0,0,2.10,-0.3,150,-0.6,1.10,1.252996,-1.071450,1.0
670,485.0,0,0,0,1,1.00,-0.9,150,-0.6,-1.10,1.252996,1.071450,1.5


In [206]:
right

,_time,_lineIndex,_lineLayer,_type,_cutDirection,_yCenter,_xCenter,_bpm,_xMovement,_yMovement,_totMovement,_angleChange,_timeChange
1,21.0,2,0,1,1,1.00,0.3,150,0.0,0.00,0.000000,NaN,0.0
2,22.0,3,1,1,5,1.55,0.9,150,0.6,0.55,0.813941,0.741947,1.0
4,23.0,1,0,1,6,1.00,-0.3,150,-1.2,-0.55,1.320038,0.429762,1.0
6,24.0,3,1,1,5,1.55,0.9,150,1.2,0.55,1.320038,0.429762,1.0
8,25.0,1,0,1,6,1.00,-0.3,150,-1.2,-0.55,1.320038,0.429762,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,480.0,3,1,1,0,1.55,0.9,150,0.0,0.55,0.550000,1.570796,1.0
665,482.0,1,0,1,6,1.00,-0.3,150,-1.2,-0.55,1.320038,0.429762,2.0
667,483.0,2,2,1,0,2.10,0.3,150,0.6,1.10,1.252996,1.071450,1.0
669,484.0,3,0,1,1,1.00,0.9,150,0.6,-1.10,1.252996,-1.071450,1.0


In [207]:
time = 0
total_distance = 0
left_time = left['_angleChange'].sum()
left_distance = left['_totMovement'].sum()
right_time = right['_angleChange'].sum()
right_distance = right['_totMovement'].sum()







df['timeChangeBefore'] = df.loc[:, ['_time']].diff().fillna(0)
df['timeChangeAfter'] = df.loc[:, ['_time']].diff(periods = -1).fillna(0)
df_bombs = df[df['_type'] == 3]
df_bombDiffBefore = df_bombs[df_bombs['timeChangeBefore'] > 0]
df_bombDiffBefore.dropna()
df_bombDiffAfter = df_bombs[df_bombs['timeChangeAfter'] > 0]
df_bombDiffAfter.dropna()
minReactTimeBefore = df_bombDiffBefore['timeChangeBefore'].min()
minReactTimeAfter = df_bombDiffAfter['timeChangeAfter'].min()

right_speed = right_distance / right_time
left_speed = left_distance / left_time

In [208]:
print('The shortest reaction time before a bomb is ' + str(minReactTimeBefore))
print('The shortest reaction time after a bomb is ' + str(minReactTimeAfter))
print('The average right hand speed is ' + str(right_speed))
print('The average left hand speed is ' + str(left_speed))

The shortest reaction time before a bomb is nan
The shortest reaction time after a bomb is nan
The average right hand speed is 2.2021976477197023
The average left hand speed is -4.008716146637433
